# Feature Selection
---

<div class="alert alert-block alert-danger">
<b>Warning:</b> Do not change this script
</div>

In [1]:
# import libraries
import pandas as pd
import re

In [2]:
path = "/home/jupyter/agra/source code/sentiment analytics/"

In [3]:
df = pd.read_csv(path+"data/resto-labelling-2.csv")

In [4]:
df.head()

,text,filtering,casefolding,check_exaggeration,repair_exaggeration,tokenize,stopword,formalisasi,stemming,polarity_score,polarity
0,"Pickup #kopi , tetap buka meski tanggal libur ...",Pickup kopi tetap buka meski tanggal libur Ja...,pickup kopi tetap buka meski tanggal libur ja...,False,pickup kopi tetap buka meski tangal libur jan ...,"['pickup', 'kopi', 'tetap', 'buka', 'meski', '...","['pickup', 'buka', 'tangal', 'libur', 'ruko', ...","['pickup', 'buka', 'tangal', 'libur', 'rumah t...","['pickup', 'buka', 'tangal', 'libur', 'rumah t...",3,positive
1,@tanyarlfes Part time di kopi kenangan aja ada...,tanyarlfes Part time di kopi kenangan aja ada ...,tanyarlfes part time di kopi kenangan aja ada ...,False,tanyarlfes part time di kopi kenangan aja ada ...,"['tanyarlfes', 'part', 'time', 'di', 'kopi', '...","['part', 'time', 'kalo', 'gasalah', 'khusus', ...","['part', 'time', 'kalau', 'tidak salah', 'khus...","['part', 'time', 'kalau', 'tidak salah', 'khus...",-1,negative
2,"aku tadi keliling"" naik motor sambil dengerin ...",aku tadi keliling naik motor sambil dengerin r...,aku tadi keliling naik motor sambil dengerin r...,False,aku tadi keliling naik motor sambil dengerin r...,"['aku', 'tadi', 'keliling', 'naik', 'motor', '...","['keliling', 'motor', 'dengerin', 'radiohead',...","['keliling', 'motor', 'dengerin', 'radiohead',...","['keliling', 'motor', 'dengerin', 'radiohead',...",5,positive
3,"wah, kopi kenangan nya sudah selesai. Makasih ...",wah kopi kenangan nya sudah selesai Makasih ko...,wah kopi kenangan nya sudah selesai makasih ko...,False,wah kopi kenangan nya sudah selesai makasih ko...,"['wah', 'kopi', 'kenangan', 'nya', 'sudah', 's...","['selesai', 'makasih', 'pahit', 'manis', 'hari...","['selesai', 'terima kasih', 'pahit', 'manis', ...","['selesai', 'terima kasih', 'pahit', 'manis', ...",10,positive
4,Menutup tahun dengan kenangan.\nKopi kenangan ...,Menutup tahun dengan kenanganKopi kenangan dan...,menutup tahun dengan kenangankopi kenangan dan...,False,menutup tahun dengan kenangankopi kenangan dan...,"['menutup', 'tahun', 'dengan', 'kenangankopi',...","['menutup', 'kenangankopi', 'konser', 'lys', '...","['menutup', 'kenangankopi', 'konser', 'lys', '...","['tutup', 'kenangankopi', 'konser', 'lys', 'ht...",-2,negative


### Preparation

In [5]:
def filtering(text):
    text = re.sub('[0-9]+', '', str(text)) #removing numberic value
    text = re.sub(r'#', '', text) #removing '#' symbol
    text = re.sub(r'[\n]+', '', text) # remove new line
    text = re.sub(r"^\s+|\s+$", "", text) #remove leading and trailing spaces in a word using OR sign to delete both
    text = re.sub(r" +", " ", text) #remove multiple space betwen words
    text = re.sub('https? :\/\/\S+', '', text) #removing hyperlink / URL
    text = re.sub(r"\b[a-zAZ]\b", "", text) #removing single char
    text = re.sub('\s+',' ',text) #removing multiple whitespace
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"") #remove tab, new line, and back slice
    text = re.sub(r'[^\w\s]', '', text) #remove puntuation& emoji (remove all besides \w > word dan \s > space)
    text = re.sub(r'(.)1+', r'1', text) # remove repeating character
    text = re.sub("[^a-zA-Z]",' ',text)
    
    return text

## Start Feature Engineering
---

In [6]:
%time df['final_text'] = df['stemming'].apply(filtering)

CPU times: user 384 ms, sys: 3.66 ms, total: 387 ms
Wall time: 387 ms


In [7]:
cols = ["text","polarity_score"]
df = df[cols]

In [8]:
df["degree_of_polarity"] = None
for i, row in df.iterrows():
    if row["polarity_score"] > 0:
        df.loc[i, "degree_of_polarity"] = 2
    elif row["polarity_score"] < 0:
        df.loc[i, "degree_of_polarity"] = 0
    else:
        df.loc[i, "degree_of_polarity"] = 1

In [9]:
cols = ["text","degree_of_polarity"]
df = df[cols]

In [10]:
df

,text,degree_of_polarity
0,"Pickup #kopi , tetap buka meski tanggal libur ...",2
1,@tanyarlfes Part time di kopi kenangan aja ada...,0
2,"aku tadi keliling"" naik motor sambil dengerin ...",2
3,"wah, kopi kenangan nya sudah selesai. Makasih ...",2
4,Menutup tahun dengan kenangan.\nKopi kenangan ...,0
...,...,...
9497,@bumblesupremacy Kopi kenangan enak tuh di grab,2
9498,Awali pagi hari mu dengan se cangkir kopi hita...,0
9499,"Kopi yang pahit bisa meninggalkan ketenangan ,...",2
9500,"ahaha valid bgt nih😭 bikin bingung gaksi, haru...",0


## Simple Analysis

In [23]:
pd.set_option('display.max_colwidth', 3000)
positive_tweet = df[df['degree_of_polarity'] == 2]
positive_tweet = positive_tweet[['final_text', 'degree_of_polarity']].reset_index(drop = True)
# positive_tweet.index += 1
positive_tweet[20:40]

,final_text,degree_of_polarity
20,indocringe enak atau nga asiang,2
21,pagi kura manis senyumuku seduh hangat dekap puan jebak nyaman hangat kenangantapi aku tahu dar temanpelangipuisi,2
22,bankneocomerce outlet sushi dapur solo stret boba bobo box orient martabak ayam gedebuk yuba tea vilo gildak trafic bun hati tawan waroeng muh chicha san chen fauzan opensea ascendex,2
23,yuniperes cinta,2
24,kh notodiputro hanajoe banyak gula mashehehebahaya abang sehat si joni,2
25,hanajoe harga cangkir kesan ukur reguler gelas minum kopi bolak kenanganandaikan buka kedai jerman,2
26,ngomongngomong americano enak juga htpscojoqlsqcn,2
27,ruia asik gtlt tpin nga mantan,2
28,momslumayan makan enak piza hut marugame udon dl htpscoyxqhmhfim,2
29,jogmfs vote,2


In [25]:
pd.set_option('display.max_colwidth', 3000)
negative_tweets = df[df['degree_of_polarity'] == 0]
negative_tweets = negative_tweets[['final_text', 'degree_of_polarity']].reset_index(drop = True)
negative_tweets.index += 1
negative_tweets[0:20]

,final_text,degree_of_polarity
1,part time kalau tidak salah khusus mahasiswa,0
2,tutup kenangankopi konser lys htpscostpsoesr,0
3,moga jumpa deh jenis tangkap muat orang nak try diri nak kenal someone be serious lasekodar tb minum habiskbye,0
4,kondisi flu minum ngerjain airdrop srat srot srat srot htpscozogrdyzcf,0
5,terima kasih kumpul orang taken temenin si tidak punya pacar fod date date date lai tutup menu moga cringe anying gp tetep cayang maaf dosa saya htpscogclyibkum,0
6,minum kopi diskon sampai besok net of kalau pakai qris bca htpscomknwptge,0
7,hai cofeipoh ini ramai hantu lah deh bukak cawangan kat,0
8,dutch chocolate enak sangat deh,0
9,bye janjiw is my friend now,0
10,sogok mantan jaga rumah ketemu pacar random sangat pulang kerja bawa tulisanya bgtu htpscotroviayhv,0


In [11]:
df.to_csv(path + "data/resto-feature-engineering-final2.csv", index=False, header=False)